<a href="https://colab.research.google.com/github/tankiet14102001/PM2.5-visualization_2D/blob/main/PM2_5_visualization_2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import os

# CHANGE THIS BELOW PATHS TO YOUR OWN DRIVE PATH CONTAINED YOUR FOLDER DATASET
path = '/content/gdrive/My Drive/Colab Notebooks/projects/thesis_dust-box_from-20240106/data/BKbox/'
os.chdir(path)
os.listdir()  # current folder structure in top layer

Mounted at /content/gdrive


['2023-딱2-01.txt',
 '2024-01-08 (1).txt',
 '2023-11-24.txt',
 '2023-11-25.txt',
 '2023-11-29.txt',
 '2023-12-02_DH.txt',
 '2023-11-30.txt',
 '2023-12-02.txt',
 '2023-11-27.txt',
 '2023-11-28.txt',
 '2023-11-26.txt',
 '2023-12-05.txt',
 '2023-12-07.txt',
 '2023-12-15.txt',
 '2023-12-14.txt',
 '2023-12-17.txt',
 '2023-12-21.txt',
 '2023-12-22.txt',
 '2023-12-24.txt',
 '2024-01-01.txt',
 '2023-12-26.txt',
 '2024-01-04.txt',
 '2024-01-05.txt',
 '2024-01-08.txt',
 '2024-01-11.txt',
 '2024-01-12.txt',
 '2024-01-05_1.txt',
 '2024-01-05_2.txt',
 '2024-01-05_3.txt',
 '2024-01-05_4.txt',
 '2024-01-12_1.txt',
 '2024-01-12_2.txt',
 '2024-01-12_3.txt',
 '2024-01-12_4.txt',
 'cleaned_data_file.txt',
 'filtered_data',
 '2024-01-15.txt',
 '2024-01-22.txt',
 '2024-01-26.txt',
 '2024-02-04.txt',
 '2024-01-01_filtered_data_map.html']

In [ ]:
import pandas as pd
import re

file_name = '2023-12-26';

# Define the wanted columns
#wanted_columns = ['DateTime', 'Temp', 'Humi', 'PM1_0', 'PM2_5', 'PM10', 'Lat', 'Long']
wanted_columns = ['DateTime', 'Temp', 'Humi', 'PM1_0', 'PM2_5', 'PM10', 'Lat', 'Long', 'Alt']

# Load the data into a DataFrame
try:
  df = pd.read_csv(file_name + '.txt', sep=',', encoding='latin1', error_bad_lines=False)
except pd.errors.ParserError as e:
  print("ParserError:", e)
  df = pd.DataFrame() # Create an empty DataFrame in case of error

print(df)

if not df.empty:
    # Select only the wanted columns
    if set(wanted_columns).issubset(df.columns):
        df = df[wanted_columns]
    else:
        missing_columns = set(wanted_columns) - set(df.columns)
        print(f"Missing columns: {missing_columns}")

"""
file_name = '2023-12-07'
lines = []
with open(file_name + '.txt', 'r', encoding='latin1') as file:
    for line in file:
        try:
            lines.append(line)
        except Exception as e:
            print(f"Error processing line: {e}")

# Create a DataFrame from the processed lines
df = pd.DataFrame(lines)
df = pd.read_csv(pd.Series(df[0]), sep=',')
"""
"""

import pandas as pd
"""
"""

file_name = '2023-12-05';

# Load data from the CSV file with error handling
try:
    df = pd.read_csv(file_name + '.txt', sep=',', encoding='latin1')
except pd.errors.ParserError as e:
    print("ParserError:", e)

"""


"""
# Select only the wanted columns
df = df[wanted_columns]
"""

# Define a function to check for invalid characters in a row
def has_invalid_characters(row):
    for value in row:
        if re.search(r'[^a-zA-Z0-9:.,\-_ ]', str(value)):
            return True
    return False

# Filter out rows with invalid characters
df = df[~df.apply(has_invalid_characters, axis=1)]

print(df)
# Print the resulting DataFrame
print(df.head())

<ipython-input-2-7dbe65aa38aa>:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_name + '.txt', sep=',', encoding='latin1', error_bad_lines=False)


    DeviceName             DateTime  Temp  Humi     Press PM1_0 PM2_5 PM10  \
0      SPARC-1  2023-12-26T07:08:38  24.3  63.9  1.001226    24    35   43   
1      SPARC-1  2023-12-26T07:08:45  24.4  62.0  1.001227    26    35   44   
2      SPARC-1  2023-12-26T07:08:52  24.4  61.9  1.001227    24    34   42   
3      SPARC-1  2023-12-26T07:08:58  24.4  61.8  1.001224    23    35   39   
4      SPARC-1  2023-12-26T07:09:05  24.5  61.7  1.001226    23    35   39   
..         ...                  ...   ...   ...       ...   ...   ...  ...   
537    SPARC-1  2023-12-26T08:10:25  31.0  42.6  1.001912    12    17   18   
538    SPARC-1  2023-12-26T08:10:32  31.0  42.5  1.001889    13    18   20   
539    SPARC-1  2023-12-26T08:10:39  31.0  42.4  1.001864    22    29   32   
540   §SPARC-1  2023-12-26T08:10:6  31.0  42.1  1.001849    32    45   55   
541    SPARC-1  2023-12-26T08:10:53  31.1  42.0  1.001832    35    51   61   

          Bat        Lat        Long         Alt  
0    1.91678

In [ ]:
# Define the directory and file name for the new file
output_directory = '/content/gdrive/My Drive/Colab Notebooks/projects/thesis_dust-box_from-20240106/data/BKbox/filtered_data/'
output_file_name = file_name + '_filtered_data.txt'

# Save the filtered DataFrame to a new CSV file
df.to_csv(output_directory + output_file_name, index=False)

print("Filtered data has been saved to:", output_directory + output_file_name)


Filtered data has been saved to: /content/gdrive/My Drive/Colab Notebooks/projects/thesis_dust-box_from-20240106/data/BKbox/filtered_data/2023-12-26_filtered_data.txt


In [ ]:
import folium
import pandas as pd

# Define the AQI dictionary
aqi = {
    'Good': {'pm2.5': [0, 12], 'color': '#006400'},
    'Moderate': {'pm2.5': [12.1, 35.4], 'color': '#FFD700'},
    'Unhealthy for Sensitive Groups': {'pm2.5': [35.5, 55.4], 'color': '#FF8C00'},
    'Unhealthy': {'pm2.5': [55.5, 150.4], 'color': 'red'},
    'Very Unhealthy': {'pm2.5': [150.5, 250.5], 'color': 'purple'},
    'Hazardous': {'pm2.5': [250.5, 500.4], 'color': 'maroon'}
}
"""
# Load data from the CSV file with error handling
try:
    df = pd.read_csv('2023-12-05.txt', sep=',', encoding='latin1')
except pd.errors.ParserError as e:
    print("ParserError:", e)
"""

"""
# Keep only the desired columns
df = df[['DateTime', 'Temp', 'Humi', 'PM2_5', 'Lat', 'Long', 'Alt']]
"""

# Remove rows with invalid PM2_5 data
df = df[df['PM2_5'].apply(lambda x: str(x).replace('.', '').isdigit())]
df['PM2_5'] = pd.to_numeric(df['PM2_5'], errors='coerce')

# Convert latitude and longitude columns to numeric
df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
df['Long'] = pd.to_numeric(df['Long'], errors='coerce')

# Remove rows with NaN values in Lat and Long
df = df.dropna(subset=['Lat', 'Long'])

# Create a map centered at the mean latitude and longitude of your data
map_center = [df['Lat'].mean(), df['Long'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)



# Add markers for each data point with PM2.5 data displayed inside the circle marker
for index, row in df.iterrows():
    # Determine the AQI category based on PM2.5 value
    pm2_5 = row['PM2_5']
    aqi_category = None
    for category, data in aqi.items():
        if pm2_5 >= data['pm2.5'][0] and pm2_5 <= data['pm2.5'][1]:
            aqi_category = category
            break

    # Determine marker color based on AQI category
    marker_color = aqi[aqi_category]['color']

    # Add CircleMarker with the specified radius
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=5,  # Adjust the size of the marker
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=1,
        popup=f"PM2.5: {pm2_5}, AQI: {aqi_category}"
    ).add_to(mymap)

    # Add text to display PM2.5 value inside the circle marker
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.DivIcon(
            icon_size=(50, 50),
            icon_anchor=(25, 25),
            #html=f"<div style='display: flex; justify-content: center; align-items: center; width: 100%; height: 100%; color: black; font-size: 8px;'>{pm2_5}</div>",
        )
    ).add_to(mymap)

# Add a legend for AQI categories
legend_html = """
     <div style="position: fixed;
                 bottom: 50px; left: 50px; width: 200px; height: 200px;
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white; opacity: 0.9;
                 ">
     &nbsp; AQI Legend <br>
     &nbsp; Good: <i class="fas fa-circle" style="color:#006400; font-size: 24px;"></i><br>
     &nbsp; Moderate: <i class="fas fa-circle" style="color:#FFD700; font-size: 24px;"></i><br>
     &nbsp; Unhealthy for Sensitive Groups: <i class="fas fa-circle" style="color:#FF8C00; font-size: 24px;"></i><br>
     &nbsp; Unhealthy: <i class="fas fa-circle" style="color:red; font-size: 24px;"></i><br>
     &nbsp; Very Unhealthy: <i class="fas fa-circle" style="color:purple; font-size: 24px;"></i><br>
     &nbsp; Hazardous: <i class="fas fa-circle" style="color:maroon; font-size: 24px;"></i><br>
      </div>
     """
mymap.get_root().html.add_child(folium.Element(legend_html))

# Save the map as an HTML file
mymap.save("/content/gdrive/My Drive/Colab Notebooks/projects/thesis_dust-box_from-20240106/graph/" + file_name + "_filtered_data_map.html")


<ipython-input-4-2bf89ac340d1>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PM2_5'] = pd.to_numeric(df['PM2_5'], errors='coerce')
<ipython-input-4-2bf89ac340d1>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
<ipython-input-4-2bf89ac340d1>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [ ]:
# Create a map centered at a location within South East Asia
map_center_sea = [10.0, 105.0]  # Choose a suitable latitude and longitude for the center
mymap_sea = folium.Map(location=map_center_sea, zoom_start=5)  # Adjust the zoom level as needed

# Add markers for each data point with PM2.5 data displayed inside the circle marker
for index, row in df.iterrows():
    # Determine the AQI category based on PM2.5 value
    pm2_5 = row['PM2_5']
    aqi_category = None
    for category, data in aqi.items():
        if pm2_5 >= data['pm2.5'][0] and pm2_5 <= data['pm2.5'][1]:
            aqi_category = category
            break

    # Determine marker color based on AQI category
    marker_color = aqi[aqi_category]['color']

    # Define custom HTML content for the popup
    popup_content = """
    <div style="max-width: 300px;">
        <b>DateTime:</b> {}<br>
        <b>PM2.5:</b> {}<br>
        <b>AQI:</b> {}<br>
        <b>Temperature:</b> {}<br>
        <b>Humidity:</b> {}<br>
        <b>Altitude:</b> {}<br>
    </div>
    """.format(row['DateTime'], pm2_5, aqi_category, row['Temp'], row['Humi'], row['Alt'])

    # Add CircleMarker with the specified radius
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=5,  # Adjust the size of the marker
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=1,
        popup=folium.Popup(popup_content, max_width=300),
    ).add_to(mymap_sea)

    # Add text to display PM2.5 value inside the circle marker
    # Add text to display PM2.5 value near the circle marker
    # Add text to display PM2.5 value above the circle marker
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.DivIcon(
            icon_size=(0, 0),  # Set icon size to zero to hide the marker
            #html=f"<div style='color: black; font-size: 8px; position: relative; bottom: 20px; left: 50%; transform: translateX(-50%); display: inline-block;'>{pm2_5}</div>",
        )
    ).add_to(mymap_sea)



# Add a legend for AQI categories
legend_html = """
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 300px;
            border: 2px solid grey; z-index:9999; font-size:14px;
            background-color:white; opacity: 0.9;">
    <div style="padding-left: 10px;">
        <b>AQI Legend</b>
    </div>
    <ul style="list-style-type: none; padding-left: 20px; margin: 0;">
        <li><i class="fa fa-circle" style="color:#006400"></i>&nbsp; Good</li>
        <li><i class="fa fa-circle" style="color:#FFD700"></i>&nbsp; Moderate</li>
        <li><i class="fa fa-circle" style="color:#FF8C00"></i>&nbsp; Unhealthy for Sensitive Groups</li>
        <li><i class="fa fa-circle" style="color:red"></i>&nbsp; Unhealthy</li>
        <li><i class="fa fa-circle" style="color:purple"></i>&nbsp; Very Unhealthy</li>
        <li><i class="fa fa-circle" style="color:maroon"></i>&nbsp; Hazardous</li>
    </ul>
</div>
"""
mymap_sea.get_root().html.add_child(folium.Element(legend_html))

# Save the map as an HTML file
mymap_sea.save("/content/gdrive/My Drive/Colab Notebooks/projects/thesis_dust-box_from-20240106/graph/" + file_name + "_filtered_data_map_sea.html")